In [1]:
import pandas as pd
import numpy as np
from statistics import mean,stdev
from scipy.spatial import distance
import math

## Function to identify Shapelets
### ts_df = Dataset
### rep = Representative Time Series
### subsequence_sample = Length of the shapelets
### rep_labels = Labels of representative time series

In [2]:
Shapelet=[]
Earliness=[]
def identifyshapelets(ts_df,rep,subsequence_sample,rep_labels):
#     rep=rep_df.drop(rep_df.columns[0],axis=1)
#     print('here')
#     rep_labels=rep_df[0]
#     print('here')
    for row in range(0,rep.shape[0]):
        ts=rep.iloc[row]
#         print(ts)
        for length in subsequence_sample:
            for j in range(0,(len(ts)-length)+1):
                ts_copy=list(ts[j:(j+length)])
#                 print('shapelet candidate',ts_copy)
#                 print('LABEL is', rep_labels[row])
                computeDistance(ts_copy,ts_df,rep_labels[row],j+length)
    print('*********************************')
    print('Shapelets Identified')
    return Shapelet, Earliness

## Function to compute distance between shapelet and timeseries subsequence
### s = subsequence
### D = Dataset
### label = label of representative time series
### last = end timestamp of the subsequence

In [3]:
def computeDistance(s,D,label,last):
#     print('in ComputeDistance')
#     print(D.head(3))
#     print('NEXT')
    labels=D['0']
    D=D.drop(D.columns[0],axis=1)
#     print(D.head(3))
    in_classdis=[]
    out_classdis=[]
    
    dis=np.full(D.shape[0],10000)#Line 1 in Table 4
    dis=dis.astype('float64')
    s_copy=list(s)
    s_copy=znorm(s_copy)
    for i in range(0,D.shape[0]):
        timeseries=D.loc[i]
        for j in range(0,(len(timeseries)-len(s)+1)):
            timeseries_copy=timeseries[j:(j+len(s))].copy()
            z=znorm(timeseries_copy)# Line 6 in Table 4
            d=distance.euclidean(z,s_copy)
            if float(d)<float(dis[i]):
                minimum=float(d)
            else:
                minimum=float(dis[i])
            dis[i]=float(minimum)
        if labels[i]==label:
#             print('Timeseries',i)
#             print('labels are equal')
            in_classdis.append(dis[i]/math.sqrt(len(s)))
        else:
#             print('Timeseries',i)
#             print('labels are not equal')
            out_classdis.append(dis[i]/math.sqrt(len(s)))
    quartile(in_classdis,out_classdis,s,last,len(s))

In [4]:
# function to perform z-normalization
def znorm(ser):
    new_ser=[]
    u=mean(ser)
    sigma=np.std(ser)
    index=0
    for value in ser:
        new_ser.append((value-u)/sigma)
        index=index+1
    new_ser=np.array(new_ser)
    return new_ser 

## Function to check the 1st and 3rd quartile
### inclass = inclass distance list
### outclass = outclass distance list
### s = candidate shapelet

In [5]:
def quartile(inclass,outclass,s,end_timestamp,sub_length):
    if float(np.percentile(inclass,75))>float(np.percentile(outclass,25)):
        Shapelet.append(s)
        Earliness.append(end_timestamp/sub_length)